# Homography estimation for Kornia DoG implementation

### Performed on 4 datasets

* boat and bark datasets from https://www.robots.ox.ac.uk/~vgg/data/affine/
* pure rotations of the first image from "bark" by multiples of 90 degrees
* pure scaling of the first image from "bark" by multiples of 0.1 adjusted so that the aspect ratio is preserved


# Comparison of Kornia baseline and fixed version

* see https://github.com/kornia/kornia/pull/2105/

```
Without fix:                                         With fix:


running experiment: synthetic rotation by multiples of pi/2

rotation        error   tentatives        inliers       rotation        error    tentatives       inliers
   90deg     0.516270         6322           4857          90deg     0.000651          6656          6617
  180deg     0.674599         5908           3802         180deg     0.001429          4436          4223
  270deg     0.454714         6170           3872         270deg     0.001259          6114          5935
     Sum     1.645583        18400          12531            Sum     0.003339         17206         16775


running experiment: bark

                error   tentatives        inliers       rotation        error    tentatives       inliers
img2         1.257790          267             49           img2     1.379990           331            91
img3         2.212071          121             28           img3     1.492706           125            48
img4         1.350093          362            193           img4     1.001385           335           252
img5         0.710791          210             90           img5     0.699119           233           171
img6         1.352123           89             43           img6     0.987482            77            62
 Sum         6.882868         1049            403            Sum     5.560682          1101           624


running experiment: boat

              error   tentatives        inliers                         error    tentatives       inliers
img2       0.625754         1640            425             img2     0.257128          1806           628
img3       0.313360         1155            345             img3     0.387596          1130           417
img4       1.635418          374             90             img4     0.834250           373           106
img5       0.868519          193             42             img5     1.171891           227            59
img6       6.004359           63             16             img6     4.849470            55            16
 Sum       9.447409         3425            918              Sum     7.500336          3591          1226


running experiment: synthetic rescaling lanczos

scale         error   tentatives        inliers            scale        error    tentatives       inliers
  0.2      0.337641           46             28              0.2     0.063100            82            69
  0.3      0.270277          219            132              0.3     0.105695           285           192
  0.4      0.316275          597            316              0.4     0.050815           697           565
  0.5      0.191226         1155            622              0.5     0.017677          1311          1160
  0.6      0.162396         1782            985              0.6     0.047982          2065          1616
  0.7      0.229305         2390           1260              0.7     0.050292          2832          2089
  0.8      0.112914         3756           2134              0.8     0.015694          4140          3694
  0.9      0.164707         4431           2340              0.9     0.051795          5233          3939
  Sum      1.784741        14376           7817              Sum     0.403049         16645         13324
```

In [1]:
from tqdm.notebook import tqdm

from dataset_utils import Hs_imgs_for_boat, Hs_imgs_for_bark, Hs_imgs_for_rotation, Hs_imgs_for_scaling
from opencv_utils import get_tentatives, get_visible_part_mean_absolute_reprojection_error


def compare_output(o1: str, o2: str):
    for line in zip(o1.split("\n"), o2.split("\n")):
        print(f"{line[0]}{line[1]:>60}")


class Printer:
    def __init__(self, print_output):
        self.print_output = print_output
        self.output = ""

    def print(self, s=""):
        if self.print_output:
            print(s)
        self.output = self.output + s + "\n"


def homography_estimation_experiment(
        detector, Hs_gt, imgs, e_name,
        instance_names, p=Printer(print_output=True), mean=True, sum=False
):

    p.print()
    p.print(f"running experiment: {e_name}")
    p.print()

    kpts_0, desc_0 = detector.detectAndCompute(imgs[0], mask=None)
    p.print(f"reference img keypoints: {len(kpts_0)}")
    p.print()
    p.print(f"{'':>8}{'error':>12}{'keypoints':>12}{'tentatives':>12}{'inliers':>10}")

    sum_reproj_err = 0.0
    sum_keypoints = 0
    sum_tent_count = 0
    sum_in_count = 0

    for other_i in tqdm(range(1, len(imgs)), leave=False):
        kpts_other, desc_other = detector.detectAndCompute(imgs[other_i], mask=None)
        kpts_n = len(kpts_other)

        src_pts, dst_pts = get_tentatives(kpts_0, desc_0, kpts_other, desc_other, ratio_threshold=0.8)
        if len(src_pts) < 4:
            print(f"WARNING: less than 4 tentatives: {len(src_pts)}")
            continue

        H_est, inlier_mask = cv.findHomography(src_pts, dst_pts,
                                               cv.RANSAC,
                                               maxIters=100000,
                                               ransacReprojThreshold=0.5,
                                               confidence=0.9999)
        H_gt = Hs_gt[other_i - 1]
        

        reproj_err = get_visible_part_mean_absolute_reprojection_error(imgs[0], imgs[other_i], H_gt, H_est)
        tent_count = len(src_pts)
        in_count = int(inlier_mask.sum())

        p.print(f"{instance_names[other_i - 1]:>8}{reproj_err:>12.6f}{kpts_n:>12}{tent_count:>12}{in_count:>10}")
        sum_reproj_err += reproj_err
        sum_keypoints += kpts_n
        sum_tent_count += tent_count
        sum_in_count += in_count

    l = len(imgs) - 1
    if mean:
        p.print(f"{'Mean':>8}{sum_reproj_err/l:>12.6f}{sum_keypoints/l:>14.1f}{sum_tent_count/l:>12.1f}{sum_in_count/l:>10.1f}")
    if sum:
        p.print(f"{'Sum':>8}{sum_reproj_err:>12.6f}{sum_keypoints:>12}{sum_tent_count:>12}{sum_in_count:>10}")


Hs_boat, imgs_boat = Hs_imgs_for_boat()
Hs_boat_rot, imgs_boat_rot = Hs_imgs_for_boat(rotate_query_imgs=True)
Hs_bark, imgs_bark = Hs_imgs_for_bark()
Hs_bark_rot, imgs_bark_rot = Hs_imgs_for_bark(rotate_query_imgs=True)
Hs_rot, imgs_rot = Hs_imgs_for_rotation()
print("Exact scale adjustments:")
Hs_scaling, imgs_scaling, scales = Hs_imgs_for_scaling()


def run_on_descriptor(descriptor, name, print_output=True, mean=True, sum=False):
    p = Printer(print_output)
    p.print()
    p.print(f"descriptor: {name}:")
    p.print()
    homography_estimation_experiment(descriptor, Hs_rot, imgs_rot, "synthetic pi rotation", ["90deg", "180deg", "270deg"], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_bark, imgs_bark, "bark", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    #homography_estimation_experiment(descriptor, Hs_bark_rot, imgs_bark_rot, "bark rotated", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_boat, imgs_boat, "boat", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    #homography_estimation_experiment(descriptor, Hs_boat_rot, imgs_boat_rot, "boat rotated", [f"img{i}" for i in range(2, 7)], p, mean, sum)
    homography_estimation_experiment(descriptor, Hs_scaling, imgs_scaling, "synthetic rescaling lanczos", [f"{s}" for s in scales], p, mean, sum)
    return None if print_output else p.output


Exact scale adjustments:
scale: 0.2 => 0.2, dimension: 765x510 => (153, 102)
scale: 0.3 => 0.2980392156862745, dimension: 765x510 => (228, 152)
scale: 0.4 => 0.4, dimension: 765x510 => (306, 204)
scale: 0.5 => 0.5019607843137255, dimension: 765x510 => (384, 256)
scale: 0.6 => 0.6, dimension: 765x510 => (459, 306)
scale: 0.7 => 0.6980392156862745, dimension: 765x510 => (534, 356)
scale: 0.8 => 0.8, dimension: 765x510 => (612, 408)
scale: 0.9 => 0.9019607843137255, dimension: 765x510 => (690, 460)


In [2]:
import cv2 as cv
import torch
from conv_quad_interp3d import FixedConvQuadInterp3d
from scale_pyramid import FixedDogScalePyramid
from kornia.feature.integrated import LocalFeature, LAFDescriptor, SIFTDescriptor
from kornia.utils import image_to_tensor
from kornia.geometry import ScalePyramid
from enum import Enum


class NumpyKorniaSiftDescriptor:

    def __init__(self, local_feature, device=torch.device('cpu')):
        self.local_feature = local_feature
        self.device = device

    @staticmethod
    def cv_kpt_from_laffs_responses(laffs, responses):
        kpts = []
        for i, response in enumerate(responses[0]):
            yx = laffs[0, i, :, 2]
            kp = cv.KeyPoint(yx[0].item(), yx[1].item(), response.item(), angle=0)
            kpts.append(kp)
        return kpts

    def detectAndCompute(self, img, mask):
        assert mask is None, "not implemented with non-trivial mask"
        if len(img.shape) == 2:
            img = img[:, :, None]
        else:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img_t = (image_to_tensor(img, False).float() / 255.).to(device=self.device)
        laffs, responses, descs = self.local_feature(img_t, mask=None)
        kpts = self.cv_kpt_from_laffs_responses(laffs, responses)
        descs = descs[0].cpu().detach().numpy()
        return kpts, descs


class CustomSIFTFeature(LocalFeature):
    """ See kornia.feature.integrated.SIFTFeature """

    def __init__(
        self,
        scale_space_detector,
        rootsift: bool = True,
        device: torch.device = torch.device('cpu'),
    ):
        patch_size: int = 41
        detector = scale_space_detector.to(device)
        descriptor = LAFDescriptor(
            SIFTDescriptor(patch_size=patch_size, rootsift=rootsift), patch_size=patch_size, grayscale_descriptor=True
        ).to(device)
        super().__init__(detector, descriptor)

    
class Version(Enum):
    FIXED = 1
    NOT_FIXED = 2
    ORIGINAL = 3


def get_scale_pyramid(version, double_image=True, min_size=15):
    if version == Version.FIXED:
        return FixedDogScalePyramid(3, 1.6, min_size=min_size, double_image=double_image, fix_upscaling=True)
    elif version == Version.NOT_FIXED:
        return FixedDogScalePyramid(3, 1.6, min_size=min_size, double_image=double_image, fix_upscaling=False)
    elif version == Version.ORIGINAL:
        return ScalePyramid(3, 1.6, min_size=min_size, double_image=double_image)
    else:
        raise ValueError(f"unexpeced version: {version}")


def get_conv_quad_interp3d(version, eps=1e-7):
    if version == Version.FIXED:
        return FixedConvQuadInterp3d(10,
                                     eps=eps,
                                     scatter_fix=True,
                                     swap_xy_fix=True)
    elif version == Version.NOT_FIXED:
        return FixedConvQuadInterp3d(10,
                                     eps=eps,
                                     scatter_fix=False,
                                     swap_xy_fix=False)
    elif version == Version.ORIGINAL:
        return ConvQuadInterp3d(10, eps=eps)
    else:
        raise ValueError(f"unexpected version: {version}")

In [16]:
from kornia.feature import BlobDoG
from kornia.geometry import ConvQuadInterp3d
from scale_space_detector import FixedScaleSpaceDetector
from kornia.feature import LAFOrienter
from kornia.feature import ScaleSpaceDetector


def get_sift_descriptor(version):

    sp = get_scale_pyramid(version, double_image=True, min_size=32)
    nms_module = get_conv_quad_interp3d(version)

    if version == Version.ORIGINAL:
        detector = ScaleSpaceDetector(
            num_features=8000,
            resp_module=BlobDoG(),
            nms_module=nms_module,
            scale_pyr_module=sp,
            ori_module=LAFOrienter(19),
            scale_space_response=True,
            minima_are_also_good=True,
            mr_size=6.0,
        )
    else:
        detector = FixedScaleSpaceDetector(
            num_features=8000,
            resp_module=BlobDoG(),
            nms_module=nms_module,
            scale_pyr_module=sp,
            ori_module=LAFOrienter(19),
            scale_space_response=True,
            minima_are_also_good=True,
            mr_size=6.0,
        )

    return NumpyKorniaSiftDescriptor(local_feature=CustomSIFTFeature(detector))


out1 = run_on_descriptor(descriptor=get_sift_descriptor(version=Version.FIXED), name="Kornia SIFT fixed", print_output=False, sum=True)
out2 = run_on_descriptor(descriptor=get_sift_descriptor(version=Version.NOT_FIXED), name="Kornia SIFT not fixed", print_output=False, sum=True)
compare_output(out1, out2)
run_on_descriptor(descriptor=get_sift_descriptor(version=Version.ORIGINAL), name="Kornia SIFT original", sum=True)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

    img2    0.625754        8000        1640       425
    img3    0.313360        8000        1155       345
    img4    1.635418        8000         374        90
    img5    0.868519        8000         193        42
    img6    6.004359        8000          63        16
    Mean    1.889482        8000.0       685.0     183.6
     Sum    9.447409       40000        3425       918

running experiment: boat rotated

reference img keypoints: 8000

               error   keypoints  tentatives   inliers


  0%|          | 0/5 [00:00<?, ?it/s]

    img2    0.976225        8000        1363       336
    img3    0.693703        8000        1042       287
    img4    1.491942        8000         395        82
    img5    1.543355        8000         181        33
    img6    5.593381        8000          64        17
    Mean    2.059721        8000.0       609.0     151.0
     Sum   10.298606       40000        3045       755

running experiment: synthetic rescaling lanczos

reference img keypoints: 8000

               error   keypoints  tentatives   inliers


  0%|          | 0/8 [00:00<?, ?it/s]

     0.2    0.337641        8000          46        28
     0.3    0.270277        8000         219       132
     0.4    0.316275        8000         597       316
     0.5    0.191226        8000        1155       622
     0.6    0.162396        8000        1782       985
     0.7    0.229305        8000        2390      1260
     0.8    0.112914        8000        3756      2134
     0.9    0.164707        8000        4431      2340
    Mean    0.223093        8000.0      1797.0     977.1
     Sum    1.784741       64000       14376      7817


In [3]:
from kornia.feature.responses import CornerHarris
from scale_space_detector import FixedScaleSpaceDetector
from kornia.feature import LAFAffineShapeEstimator, LAFOrienter
from warnings import catch_warnings


def get_harris_descriptor(version, double_image=True):

    sp = get_scale_pyramid(version, double_image=double_image, min_size=15)
    nms_module = get_conv_quad_interp3d(version, eps=2e-4)
    resp = CornerHarris(0.05)

    with catch_warnings(record=True):
        aff_module=LAFAffineShapeEstimator(patch_size=19)

    if version == Version.ORIGINAL:
        harris_affine_local_detector = ScaleSpaceDetector(num_features=8000,
                                                          resp_module=resp,
                                                          nms_module=nms_module,
                                                          mr_size=6.0,
                                                          scale_pyr_module=sp,
                                                          aff_module=aff_module,
                                                          ori_module=LAFOrienter(patch_size=19),
                                                          minima_are_also_good=False)

    else:
        harris_affine_local_detector = FixedScaleSpaceDetector(num_features=8000,
                                                               resp_module=resp,
                                                               nms_module=nms_module,
                                                               mr_size=6.0,
                                                               scale_pyr_module=sp,
                                                               aff_module=aff_module,
                                                               ori_module=LAFOrienter(patch_size=19),
                                                               minima_are_also_good=False)

    return NumpyKorniaSiftDescriptor(local_feature=CustomSIFTFeature(harris_affine_local_detector))


out1 = run_on_descriptor(descriptor=get_harris_descriptor(version=Version.FIXED), name="Kornia Harris+SIFT fixed", print_output=False)
out2 = run_on_descriptor(descriptor=get_harris_descriptor(version=Version.NOT_FIXED), name="Kornia Harris+SIFT not fixed", print_output=False)
compare_output(out1, out2)
#run_on_descriptor(descriptor=get_harris_descriptor(version=Version.ORIGINAL), name="Kornia Harris+SIFT original")


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

                                                            
descriptor: Kornia Harris+SIFT fixed:                   descriptor: Kornia Harris+SIFT not fixed:
                                                            
                                                            
running experiment: synthetic pi rotation                   running experiment: synthetic pi rotation
                                                            
reference img keypoints: 8000                               reference img keypoints: 8000
                                                            
               error   keypoints  tentatives   inliers                     error   keypoints  tentatives   inliers
   90deg    0.000086        8000        5115      5089         90deg    0.490866        8000        5296      4801
  180deg    0.001696        8000        2329      2133        180deg    0.695779        8000        5167      4853
  270deg    0.001426        8000        3647      3511      

In [ ]:
from kornia.feature.responses import BlobHessian
from kornia.feature import LAFAffineShapeEstimator, LAFOrienter
from scale_space_detector import FixedScaleSpaceDetector
from kornia.feature import ScaleSpaceDetector
import warnings


def get_hessian_descriptor(version, double_image=True):

    sp = get_scale_pyramid(version, double_image=double_image, min_size=15)
    nms_module = get_conv_quad_interp3d(version, eps=2e-4)
    resp = BlobHessian()
    with warnings.catch_warnings(record=True):
        aff_module=LAFAffineShapeEstimator(patch_size=19)

    if version == Version.ORIGINAL:
        harris_affine_local_detector = ScaleSpaceDetector(num_features=8000,
                                                          resp_module=resp,
                                                          nms_module=nms_module,
                                                          mr_size=6.0,
                                                          scale_pyr_module=sp,
                                                          aff_module=aff_module,
                                                          ori_module=LAFOrienter(patch_size=19),
                                                          minima_are_also_good=True)
    else:
        harris_affine_local_detector = FixedScaleSpaceDetector(num_features=8000,
                                                               resp_module=resp,
                                                               nms_module=nms_module,
                                                               mr_size=6.0,
                                                               scale_pyr_module=sp,
                                                               aff_module=aff_module,
                                                               ori_module=LAFOrienter(patch_size=19),
                                                               minima_are_also_good=True)
    return NumpyKorniaSiftDescriptor(local_feature=CustomSIFTFeature(harris_affine_local_detector))


out1 = run_on_descriptor(descriptor=get_hessian_descriptor(version=Version.FIXED), name="Kornia Hessian+SIFT fixed", print_output=False)
out2 = run_on_descriptor(descriptor=get_hessian_descriptor(version=Version.NOT_FIXED), name="Kornia Hessian+SIFT not fixed", print_output=False)
compare_output(out1, out2)
#run_on_descriptor(descriptor=get_hessian_descriptor(version=Version.ORIGINAL), name="Kornia Hessian+SIFT original")


Hessian detector with double image


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]